# Transfer Learning

Большинство эффективных моделей, особенно это касается победителей ImageNet, обладают одной общей и неприятной особенностью - количество их параметров измеряется десятками и сотнями миллионов. Это много. И обучение сети с такой архитектурой - долгая и ресурсоёмкая задача. Однако, это ещё не повод не использовать такие сети в своих задачах, поскольку существует Transfer Learning. 

Идея, которая лежит за этим названием, следующая. Допустим, кто-то обучил Inception v3 и получил хороший результат на ImageNet. Допустим этот кто-то настолько хороший человек, что поделился своей моделькой со всеми желающими. Но эта моделька работает только для классов, представленных в ImageNet, а мы хотим написать сеть, которая отличает крокодилов от аллигаторов. 

А теперь вспоминаем, какие признаки выделяют фильтры первых свёрточных слоёв. Это очень простые признаки - вертикальные и горизонтальные границы, диагональные линии, яркие точки на тёмном фоне и так далее. Следующие слои выделяют чуть более сложные, однако по-прежнему очень общие, признаки - элементарные формы. Почему бы нам не оставить эти слои в покое и не учить только некоторое количество последних слоёв? Это сильно уменьшит количество параметров для обучения.

Резюмируем: идея Transfer Learning заключается в том, чтобы взять хорошую модель, оставить базовые фильтры и переучить фильтры высоких порядков под свою задачу.

# Transfer Learning в Keras

## Использование предобученной модели

Готовые модели лежат в модуле `keras.applications`. Для того, чтобы посмотреть доступные модели можете написать `from keras.applications import ` и нажать на Tab, для выбора варианта автозаполнения. Возьмём для эксперимента что-нибудь полегче, например, [MobileNet](https://habr.com/ru/post/352804/). И сразу же посмотрим её архитектуру (та ещё простыня).

In [11]:
from keras.applications import MobileNet

model = MobileNet()
model.summary()

Чтобы подгрузить сразу обученную на ImageNet модель, при создании модели в качестве параметров нужно передать `weights='imagenet'`. Помимо этого можно указать параметр `include_top=False`, чтобы не подгружать последние полносвязные слои модели, используемые в качестве дискриминаторов, то есть, сопоставляющие признакам, которые выявили свёртки, класс, в данном случае один из тысячи классов ImageNet.

In [8]:
from keras.applications import MobileNet

model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# model.summary()

True

А теперь расположим её слои по номерам (заодно посмотрим, какие слои ушли с параметром include_top) и выберем, какие заморозить:

In [52]:
print('#\tName\t\t\t\tTrainable Parameters')
for i, layer in enumerate(model.layers):
    print(i, '%-22s' % layer.name, layer.count_params(), sep='\t')

#	Name				Trainable Parameters
0	input_10              	0
1	conv1_pad             	0
2	conv1                 	864
3	conv1_bn              	128
4	conv1_relu            	0
5	conv_dw_1             	288
6	conv_dw_1_bn          	128
7	conv_dw_1_relu        	0
8	conv_pw_1             	2048
9	conv_pw_1_bn          	256
10	conv_pw_1_relu        	0
11	conv_pad_2            	0
12	conv_dw_2             	576
13	conv_dw_2_bn          	256
14	conv_dw_2_relu        	0
15	conv_pw_2             	8192
16	conv_pw_2_bn          	512
17	conv_pw_2_relu        	0
18	conv_dw_3             	1152
19	conv_dw_3_bn          	512
20	conv_dw_3_relu        	0
21	conv_pw_3             	16384
22	conv_pw_3_bn          	512
23	conv_pw_3_relu        	0
24	conv_pad_4            	0
25	conv_dw_4             	1152
26	conv_dw_4_bn          	512
27	conv_dw_4_relu        	0
28	conv_pw_4             	32768
29	conv_pw_4_bn          	1024
30	conv_pw_4_relu        	0
31	conv_dw_5             	2304
32	conv_dw_5_bn          	1024
33	c

Будем считать, что хватит первых 8 свёрточных слоёв. Последний слой, получается, номер 55. Заморозим до него. Однако, [в Керасе есть одна неприятная вещь](http://blog.datumbox.com/the-batch-normalization-layer-of-keras-is-broken/), касающаяся реализации BatchNorm слоёв. 

Подробнее о баге читайте в статейке, я лишь озвучу главное следствие: **во время Transfer Learning нельзя замораживать BatchNorm слои.**

# Датасет

В качестве учебного датасета будем использовать [этот](https://github.com/kendemu/woman-man-recog). Будем определять по фотографии, кто перед нами - парень или девушка. И это хороший выбор, потому, что в ImageNet нет таких классов.

Для начала датасет нужно скачать. [Инструкция как пользоваться Google Drive в Colab и ссылки на датасеты](http://iostream.pw/how_to_drive).

In [17]:
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# import pickle

# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)



# id = '1nvStwFCTvsS_8wilNK1VTbk_x2G7hiLj'

# downloaded = drive.CreateFile({'id': id}) 
# downloaded.GetContentFile('data.pickle') 

Напишем функцию, которая разбивает датасет на обучающую и валидационную выборку и возвращает их в формате, подобному тому, что мы видели в MNIST: `(x_train, y_train), (x_val, y_val)`

In [1]:
import pickle
import numpy as np

def load_data(train_percentage=0.8):
    dataset = pickle.load(open('man_or_woman.pickle', 'rb'))
    men_train, women_train = np.random.rand(dataset['men'].shape[0]) < train_percentage, np.random.rand(dataset['women'].shape[0]) < train_percentage
    men_val, women_val = ~men_train, ~women_train

    x_train = np.concatenate([dataset['men'][men_train], dataset['women'][women_train]], axis=0)
    x_val = np.concatenate([dataset['men'][men_val], dataset['women'][women_val]], axis=0)

    y_train, y_val = np.zeros(shape=(x_train.shape[0], 1), dtype=bool), np.zeros(shape=(x_val.shape[0], 1), dtype=bool)
    y_train[:np.sum(men_train)] = 1
    y_val[:np.sum(men_val)] = 1
    y_train, y_val = np.hstack([y_train, ~y_train]), np.hstack([y_val, ~y_val])
    
    shuffled_order = np.random.choice(np.arange(x_train.shape[0]), x_train.shape[0], replace=False)
    
    return (x_train[shuffled_order], y_train[shuffled_order]), (x_val, y_val)

In [2]:
import io
import matplotlib.pyplot as plt
import numpy as np
from pyzip import PyZip

dogz = PyZip().from_file('catordogs/dogs.zip')
catz = PyZip().from_file('catordogs/cats.zip')

cats = np.array([plt.imread(io.BytesIO(catz[fname]), format='jpg') for fname in catz])
dogs = np.array([plt.imread(io.BytesIO(dogz[fname]), format='jpg') for fname in dogz])

def load_data():
    dogs_train, cats_train = np.random.rand(dogs.shape[0]) < 0.9, np.random.rand(cats.shape[0]) < 0.9
    dogs_val, cats_val = ~dogs_train, ~cats_train
    
    x_train = np.concatenate([dogs[dogs_train], cats[cats_train]], axis=0)
    x_val = np.concatenate([dogs[dogs_val], cats[cats_val]], axis=0)
    
    y_train, y_val = np.zeros(shape=(x_train.shape[0], 1), dtype=bool), np.zeros(shape=(x_val.shape[0], 1), dtype=bool)
    y_train[:np.sum(dogs_train)] = 1
    y_val[:np.sum(dogs_val)] = 1
    y_train, y_val = np.hstack([y_train, ~y_train]), np.hstack([y_val, ~y_val])
    
    shuffled_order = np.random.choice(np.arange(x_train.shape[0]), x_train.shape[0], replace=False)
    return (x_train[shuffled_order], y_train[shuffled_order]), (x_val, y_val)

In [13]:
from keras.applications import MobileNet
from keras import Model
from keras.layers import Convolution2D, Reshape, Dropout, Softmax, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy

# Подгружаем модель с весами
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Записываем её выход
x = model.output
# И, посольку он является выходом свёртки, сделаем его плоским
x = Flatten()(x)

# Напишем дискриминатор в виде автоэнкодера (несколько полносвязных слоёв с понижением количества элементов):
x = Dense(400, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(20, activation='relu')(x)
x = Dropout(0.1)(x)

# И выходной слой на два класса:
out = Dense(2, activation='softmax')(x)

# После чего соберём нашу новую модель, указав её вход и новый выход:
model = Model(inputs=model.input, outputs=out)

# А теперь сам Transfer Learning. Чтобы заморозить слои достаточно просто выставить их атрибут .trainable в False
for N in range(0):
    layer = model.layers[N]
    is_not_batchnorm = not isinstance(model.layers[3], BatchNormalization)
    layer.trainable = False if is_not_batchnorm else True  # Не забываем про то, что нельзя замораживать batchnorm

# Learning rate возьмём ниже, чем обычно - мы ведь доучиваем. Всё остальное - как обычно. Ииии обучаем.
optimizer = Adam(lr=0.0001, amsgrad=True)  
loss = binary_crossentropy
metrics = [binary_accuracy]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

(x_train, y_train), (x_val, y_val) = load_data(0.7)

model.fit(x_train, y_train,
          validation_data=(x_val, y_val), 
          epochs=40, 
          batch_size=32, 
          shuffle=True)

Train on 1002 samples, validate on 404 samples
Epoch 1/40
1002/1002 [==============================] - 13s 13ms/step - loss: 0.6737 - binary_accuracy: 0.7226 - val_loss: 0.2144 - val_binary_accuracy: 0.9183
Epoch 2/40
1002/1002 [==============================] - 5s 5ms/step - loss: 0.2383 - binary_accuracy: 0.9032 - val_loss: 0.3265 - val_binary_accuracy: 0.8787
Epoch 3/40
1002/1002 [==============================] - 5s 5ms/step - loss: 0.1740 - binary_accuracy: 0.9381 - val_loss: 0.1718 - val_binary_accuracy: 0.9233
Epoch 4/40
1002/1002 [==============================] - 5s 5ms/step - loss: 0.1571 - binary_accuracy: 0.9431 - val_loss: 0.5708 - val_binary_accuracy: 0.8762
Epoch 5/40
1002/1002 [==============================] - 5s 5ms/step - loss: 0.0393 - binary_accuracy: 0.9850 - val_loss: 0.5429 - val_binary_accuracy: 0.8886
Epoch 6/40
1002/1002 [==============================] - 5s 5ms/step - loss: 0.0264 - binary_accuracy: 0.9860 - val_loss: 0.3562 - val_binary_accuracy: 0.9134
Epo

KeyboardInterrupt: 

И наконец проверим, что напредсказывала наша обученная модель:

In [6]:
import matplotlib.pyplot as plt
gender = {0: 'M', 1: 'F'}

pred = model.predict(x_val)
predicted_class = np.argmax(pred, axis=1) 

match = predicted_class == np.argmax(y_val, axis=1)
mismatch = ~match
print(f'Total mismatches: {mismatch.sum()}; Accuracy = {match.mean() * 100}%')

x = x_val[mismatch]
y = np.argmax(y_val, axis=1)[mismatch]
p = predicted_class[mismatch]

fig, axes = plt.subplots(4, 6, figsize=(24, 16));
samples = np.random.choice(np.arange(y.shape[0]), axes.size, replace=False)
for i, ax in zip(samples, axes.ravel()):
    ax.imshow(x[i], cmap='gray_r')
    ax.set_title(f'Predicted as {gender[p[i]]}. True labels is {gender[y[i]]}')
    
plt.show();

Total mismatches: 35; Accuracy = 91.13924050632912%
(35,)


<Figure size 2400x1600 with 24 Axes>